# Neural Networks Street View Housing Number Digit Recognition
The data is image data of google street view with single digit.

## Description
SVHN is a real-world image dataset for developing machine learning and object recognition algorithms with the minimal requirement on data formatting but comes from a significantly harder, unsolved, real-world problem (recognizing digits and numbers in natural scene images). SVHN is obtained from house numbers in Google Street View images

In this assignment, we will work on the stock prices dataset named "prices.csv". Task is to create a Neural Network to classify closing price for a stock based on some parameters.

In [1]:
# Initialize the random number generator
import random
random.seed(0)

# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

### Load the data
- load the svhn data file from google drive


In [1]:
# run this cell to upload file using GUI if you are using google colab

#from google.colab import files
#files.upload()

In [2]:
# run this cell to to mount the google drive if you are using google colab

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import BatchNormalization, Dropout, Input, Flatten
from keras import optimizers


In [4]:
h5f=h5py.File('/content/drive/My Drive/Education/SVHN-NeuralNetwork/SVHN_single_grey1.h5','r')
h5f.keys()

<KeysViewHDF5 ['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']>

In [5]:
# Load the Train and Test Data set
X_train=h5f['X_train'][:]
y_train=h5f['y_train'][:]
X_test=h5f['X_test'][:]
y_test=h5f['y_test'][:]
X_val=h5f['X_val'][:]
y_val=h5f['y_val'][:]

In [6]:
# Check the shape of train/test/validation data
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)


(42000, 32, 32)
(18000, 32, 32)
(60000, 32, 32)
(42000,)
(18000,)
(60000,)


In [7]:
# Reshape the train/test/validation
X_train1=X_train.reshape(42000,1024)
X_test1=X_test.reshape(18000,1024)
X_val1=X_val.reshape(60000,1024)

In [8]:
# One hot encode labels for train/test/validation
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

### Define Model
- Initialize a Sequential model
- Add a Flatten layer
- Add a Dense layer with ten neuron (for 10 digits) as output
  - add 'Relu' as activation function
  - add 'Softmax' as output function


In [9]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
model.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32)))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())


In [10]:
# Adding Hidden Layers and Output Layer
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

### Compile the model
- Compile the model
- Use "adam" optimizer
- Use "categorical cross-entropy" for loss
- Use Accuracy for metrics

In [11]:
#Create optimizer with non-default learning rate
#sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 1024)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dense (Dense)                (None, 200)               205000    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 230,206
Trainable params: 228,158
Non-trainable params: 2,048
_________________________________________________________________


### Fit the model
- epochs: 50
- batch size: 128
- specify validation data

In [13]:
model.fit(X_train,y_train,        
          validation_data=(X_test,y_test),
          epochs=50,
          batch_size=128)

Epoch 1/50
329/329 [==============================] - 2s 6ms/step - loss: 1.3578 - accuracy: 0.5600 - val_loss: 0.9450 - val_accuracy: 0.7104
Epoch 2/50
329/329 [==============================] - 2s 6ms/step - loss: 0.8757 - accuracy: 0.7310 - val_loss: 0.8107 - val_accuracy: 0.7572
Epoch 3/50
329/329 [==============================] - 2s 6ms/step - loss: 0.7567 - accuracy: 0.7678 - val_loss: 0.7636 - val_accuracy: 0.7748
Epoch 4/50
329/329 [==============================] - 2s 6ms/step - loss: 0.6660 - accuracy: 0.7948 - val_loss: 0.7668 - val_accuracy: 0.7716
Epoch 5/50
329/329 [==============================] - 2s 5ms/step - loss: 0.6129 - accuracy: 0.8120 - val_loss: 0.6930 - val_accuracy: 0.7983
Epoch 6/50
329/329 [==============================] - 2s 5ms/step - loss: 0.5754 - accuracy: 0.8235 - val_loss: 0.6621 - val_accuracy: 0.8074
Epoch 7/50
329/329 [==============================] - 2s 5ms/step - loss: 0.5412 - accuracy: 0.8321 - val_loss: 0.6537 - val_accuracy: 0.8167
Epoch 

### Evaluate the model
- Evaluate the model on test data

In [14]:
model.evaluate(X_test,y_test)

563/563 [==============================] - 2s 3ms/step - loss: 0.7810 - accuracy: 0.8436


[0.7809785604476929, 0.8436111211776733]

In [15]:
# Validating the model output with actual
predictions = model.predict(X_test)

In [16]:
np.argmax(predictions[1870])

1

In [17]:
# Acutal value of row 1870
y_test[1870]


array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [28]:
## Recreating the model with drop out layers

#Initialize Sequential model
model1 = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
model1.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32)))

#Normalize the data
model1.add(tf.keras.layers.BatchNormalization())

### Add two fully connected layers with 200 and 100 neurons respectively with `relu` activations. Add a dropout layer with `p=0.25`

In [29]:
#Add 1st hidden layer
model1.add(tf.keras.layers.Dense(200, activation='relu'))
## Adding Batch Normalization Layer
model1.add(tf.keras.layers.BatchNormalization())
#Add 2nd hidden layer
model1.add(tf.keras.layers.Dense(100, activation='relu'))
## Adding Batch Normalization Layer
model1.add(tf.keras.layers.BatchNormalization())


In [30]:
#Add OUTPUT layer
model1.add(tf.keras.layers.Dense(10, activation='softmax'))

In [31]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_6 (Dense)              (None, 200)               205000    
_________________________________________________________________
batch_normalization_5 (Batch (None, 200)               800       
_________________________________________________________________
dense_7 (Dense)              (None, 100)               20100     
_________________________________________________________________
batch_normalization_6 (Batch (None, 100)               400       
_________________________________________________________________
dense_8 (Dense)              (None, 10)               

In [32]:
## Define Checkpoint
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath)

In [33]:
#Compile the model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
# Train the model
model1.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=50,
          batch_size=12,
             callbacks=[model_checkpoint_callback]
         )

Epoch 1/50
3500/3500 [==============================] - 20s 6ms/step - loss: 1.0977 - accuracy: 0.6526 - val_loss: 0.7276 - val_accuracy: 0.7830
Epoch 2/50
3500/3500 [==============================] - 20s 6ms/step - loss: 1.0585 - accuracy: 0.6627 - val_loss: 0.7094 - val_accuracy: 0.7942
Epoch 3/50
3500/3500 [==============================] - 20s 6ms/step - loss: 1.0230 - accuracy: 0.6744 - val_loss: 0.6842 - val_accuracy: 0.7980
Epoch 4/50
3500/3500 [==============================] - 20s 6ms/step - loss: 0.9940 - accuracy: 0.6865 - val_loss: 0.6712 - val_accuracy: 0.8032
Epoch 5/50
3500/3500 [==============================] - 20s 6ms/step - loss: 0.9767 - accuracy: 0.6916 - val_loss: 0.6322 - val_accuracy: 0.8154
Epoch 6/50
3500/3500 [==============================] - 20s 6ms/step - loss: 0.9532 - accuracy: 0.6992 - val_loss: 0.6394 - val_accuracy: 0.8082
Epoch 7/50
3500/3500 [==============================] - 20s 6ms/step - loss: 0.9394 - accuracy: 0.7013 - val_loss: 0.6052 - val_ac

In [37]:
# Accuracy with drop out layers
# Little improvement in accuracy but good reduction in loss.
test_loss,test_acc=model1.evaluate(X_test,y_test)

563/563 [==============================] - 2s 4ms/step - loss: 0.4944 - accuracy: 0.8577


In [38]:
# Validating predicting data with actual data
predictions1=model1.predict(X_test)

In [39]:
np.argmax(predictions1[1877])

1

In [40]:
y_test[1877]
# Predicted value is class 4 which matches with actual value

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)